In [27]:
import numpy as np
import pandas as pd
from rdkit import Chem
from mordred import Calculator, descriptors

In [ ]:
# загружаем исходные данные
df = pd.read_csv('6.csv')

In [29]:
# удаляем молекулы, для которых (пока что) не были найдены смайлс
df_clear = df.dropna(subset=['normalised_name'])
df_clear.shape

(4683, 11)

In [30]:
# задаем вычислитель дескрипторов
calc = Calculator(descriptors, ignore_3D=True)

In [31]:
# количество дескрипторов без использования 3d дескрипторов, то есть атомных координат
len(calc.descriptors)

1613

In [ ]:
# добавляем в датафрем колонку mol файлов, для тех соединений, для которых это возможно сделать
mols = [Chem.MolFromSmiles(smi) for smi in df_clear['normalised_name'].tolist()]
df_clear['mol_file'] = mols

In [47]:
# разбиваем датафрейм на части, которые содержат mol file и None
df_nans = df_clear.loc[df_clear.mol_file.isna()]
df_yes = df_clear.loc[df_clear.mol_file.notna()]

In [ ]:
# выгружаем дескрипторы для датафрейма, который содержит mol file
df_molecular_desc = calc.pandas(df_yes['mol_file'])
df_molecular_desc

In [57]:
# объединяем датафреймы
df_with_desc = pd.concat([df_yes, df_molecular_desc], axis=1)
df_result = pd.concat([df_with_desc, df_nans])
df_result.shape

(4683, 1625)

In [ ]:
df_result.to_csv('db_with_descriptors.csv', index=False)